# Métricas de concordância

Fontes importantes:
- Doc CSV: https://docs.python.org/3/library/csv.html
- Doc XML Element Tree: https://docs.python.org/3/library/xml.etree.elementtree.html

Esse script serve para salvar anotações em um arquivo CSV. Seu intuito é gerar uma representação de dados adequada à implementação de métricas de concordância.

### Append de arquivos xml e instanciação de lista de roots

In [1]:
import os

xmls = os.listdir('./xmls')

In [2]:
xmls

['17_133.28.6.2013_v1.xml',
 '11_243.20.11.2013_v1.xml',
 '19_152.28.7.2014_v1.xml',
 '20_147.21.7.2014_v1.xml',
 '16_167.14.8.2013_v1.xml',
 '8_200.26.9.2013_v1.xml',
 '10_193.17.9.2013_v1.xml',
 '6_191.13.9.2013_v1.xml',
 '12_244.21.11.2013_v1.xml',
 '4_2.3.1.2013_v1.xml',
 '5_56.19.3.2013_v1.xml',
 '15_166.13.8.2013_v1.xml',
 '7_188.10.9.2013_v1.xml',
 '14_204.1.10.2013_v1.xml',
 '18_119.12.6.2013_v1.xml',
 '9_202.27.9.2013_v1.xml',
 '13_212.10.10.2013_v1.xml']

In [3]:
import xml.etree.ElementTree as ET

# lista de roots para iteração posterior
roots = []

# populando lista de roots e iterando na lista xmls
for xml in xmls:
    xml = './xmls/' + xml
    tree = ET.parse(xml)
    root = tree.getroot()
    roots.append(root)

### Definição das colunas do CSV

O CSV foi montado com informações provientes do NidoTat. A ideia foi reunir todos os campos que podem vir a ser úteis na alimentação de métricas de concordância.

As colunas selecionadas foram:
- Id do DODF
- Tipo da relação
- Id da relação
- Anotador da relação
- Tipo da entidade
- Id da entidade
- Anotador da entidade
- Offset da entidade
- Length da entidade
- Texto da entidade

In [4]:
import csv

# abre csv para escrita de headers
with open('agreement.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'offset', 'length', 'texto'])

### XML -> CSV

In [5]:
from tqdm.notebook import tqdm

# abre csv para escrita em modo append
with open('agreement.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    # itera na lista de roots
    for root in tqdm(roots):
        # coleta id do dodf
        id_dodf = root.find("./document/id")
        id_dodf_text = id_dodf.text
        # cria lista de ids de relações
        ids_rel = []
        for rel in root.findall("./document/passage/relation"):
            id_rel = rel.get('id')
            ids_rel.append(id_rel)
            # coleta tipo e anotador da relação
            for info in rel.findall('infon'):
                if info.get('key') == 'type':
                    tipoRel = info.text
                    #print(tipoRel)
                elif info.get('key') == 'annotator':
                    annotatorRel = info.text
                    #print(annotatorRel)
            # cria lista de ids de anotações
            ids_anno = []
            for info in rel.findall('node'):
                id_anno = info.get('refid')
                ids_anno.append(id_anno)
            #print(ids_anno)
            # loop na lista de ids
            for id_anno in ids_anno:
                # encontra e itera sobre todos os elementos annotation do xml
                for anno in root.findall("./document/passage/annotation"):
                    # para cada anotação definida por um id, coleta o tipo, anotador, offset, length e texto
                    if anno.get('id') == id_anno:
                        # encontra tipo e anotador
                        for info in anno.findall('infon'):
                            if info.get('key') == 'type':
                                tipoAnno = info.text
                                #print(tipoAnno)
                            elif info.get('key') == 'annotator':
                                annotatorAnno = info.text
                                #print(annotatorAnno)
                        # encontra offset e length
                        for info in anno.findall('location'):
                            offset = info.get('offset')
                            #print(offset)
                            length = info.get('length')
                            #print(length)
                        # encontra texto
                        for info in anno.findall('text'):
                            texto = info.text
                            #print(texto)
                        # escreve linha no csv
                        writer.writerow([id_dodf_text, tipoRel, id_rel, annotatorRel, tipoAnno, id_anno, annotatorAnno, offset, length, texto])

  0%|          | 0/17 [00:00<?, ?it/s]

In [6]:
import pandas as pd

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


In [136]:
df.tipo_rel.value_counts()

Ato_Exoneracao_Comissionado       3891
Ato_Nomeacao_Comissionado         3851
Ato_Tornado_Sem_Efeito_Exo_Nom    2815
Ato_Retificacao_Comissionado      2691
Ato_Retificacao_Efetivo           1895
Ato_Substituicao                  1793
Ato_Exoneracao_Efetivo            1555
Ato_Cessao                         370
Ato_Nomeacao_Efetivo               197
Ato_Abono_Permanencia              175
Ato_Tornado_Sem_Efeito_Apo         170
Ato_Reversao                        89
Name: tipo_rel, dtype: int64

In [137]:
df.tipo_ent.value_counts()

nome                          2022
orgao                         1600
hierarquia_lotacao            1512
cargo_comissionado            1411
simbolo                       1358
                              ... 
matricula_siape                  3
data_dodf_edital_normativo       3
data                             3
tipo_edicao                      2
especialidade                    1
Name: tipo_ent, Length: 69, dtype: int64

In [138]:
df.anotador_rel.value_counts()

jose_reinaldo       3851
renato_nobre        3667
livia_fonseca       2564
matheus_stauffer    2413
khalil_carsten      2189
leonardo_maffei     1549
tatiana_franco      1443
pedro_henrique      1432
thiago_faleiros      384
Name: anotador_rel, dtype: int64

In [139]:
df.anotador_ent.value_counts()

DODFMiner           4798
lindeberg           2878
tatiana_franco      2428
matheus_stauffer    2083
vinicius_borges     2057
patricia_medyna     1219
livia_fonseca       1118
pedro_henrique      1066
teo_decampos         753
leonardo_maffei      716
alvesisaque          141
jose_reinaldo        129
fredguth              60
gerente_matheus       28
khalil_carsten        13
thiago_faleiros        4
renato_nobre           1
Name: anotador_ent, dtype: int64

## 1ª iteração

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


Para correlação offset/length/user, parece mais fácil extrair apenas a listagem em ordem e depois lidar com duplicatas.

In [119]:
offsets = df.offset.to_list()
offsets

[205008,
 205016,
 205078,
 205078,
 205089,
 205103,
 205103,
 205286,
 205301,
 205008,
 205016,
 205078,
 205078,
 205089,
 205103,
 205103,
 205286,
 205301,
 205008,
 205016,
 205078,
 205078,
 205089,
 205103,
 205103,
 205286,
 205301,
 208583,
 208720,
 208786,
 208573,
 208629,
 208677,
 208732,
 208753,
 205008,
 205016,
 205078,
 205078,
 205089,
 205286,
 205103,
 205840,
 205855,
 205560,
 205570,
 205611,
 205667,
 205677,
 205693,
 209173,
 209311,
 209377,
 209344,
 209163,
 209217,
 209219,
 209230,
 209268,
 209322,
 205397,
 205511,
 205526,
 205336,
 205344,
 205387,
 205411,
 206086,
 206101,
 205962,
 205895,
 205889,
 205974,
 205987,
 205987,
 206214,
 206135,
 206141,
 206225,
 206240,
 206240,
 206372,
 206387,
 206808,
 206715,
 206721,
 206819,
 206831,
 206831,
 206935,
 207088,
 206985,
 206991,
 207077,
 207101,
 207171,
 207540,
 207481,
 207490,
 207500,
 207544,
 207594,
 207722,
 208440,
 208372,
 208539,
 208317,
 208376,
 208308,
 208326,
 208450,
 

In [120]:
lengths = df.length.to_list()
lengths

[326,
 31,
 4,
 7,
 9,
 193,
 178,
 47,
 32,
 326,
 31,
 4,
 7,
 9,
 193,
 178,
 47,
 32,
 326,
 31,
 4,
 7,
 9,
 193,
 178,
 47,
 32,
 40,
 6,
 32,
 246,
 32,
 11,
 16,
 28,
 326,
 31,
 7,
 4,
 9,
 47,
 178,
 47,
 32,
 328,
 40,
 24,
 6,
 11,
 142,
 40,
 6,
 32,
 28,
 247,
 12,
 22,
 22,
 11,
 17,
 9,
 47,
 32,
 223,
 12,
 6,
 94,
 47,
 32,
 6,
 24,
 245,
 8,
 109,
 94,
 6,
 286,
 29,
 10,
 127,
 142,
 48,
 33,
 6,
 269,
 34,
 7,
 99,
 114,
 48,
 8,
 238,
 42,
 6,
 65,
 51,
 3,
 440,
 9,
 28,
 12,
 123,
 33,
 6,
 3,
 32,
 8,
 11,
 264,
 33,
 9,
 69,
 33,
 18,
 19,
 11,
 342,
 11,
 21,
 6,
 17,
 32,
 12,
 11,
 218,
 20,
 34,
 6,
 32,
 16,
 12,
 217,
 31,
 8,
 29,
 32,
 16,
 397,
 12,
 21,
 8,
 82,
 40,
 256,
 40,
 11,
 6,
 17,
 33,
 32,
 40,
 32,
 385,
 44,
 8,
 34,
 11,
 18,
 19,
 8,
 52,
 7,
 18,
 2,
 18,
 27,
 6,
 43,
 2,
 364,
 8,
 52,
 7,
 18,
 2,
 18,
 27,
 6,
 43,
 2,
 364,
 8,
 52,
 7,
 18,
 2,
 18,
 27,
 6,
 43,
 2,
 364,
 6,
 8,
 332,
 27,
 82,
 154,
 8,
 52,
 364,
 7,
 18,
 

In [121]:
texts = df.texto.to_list()
texts

['EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.',
 ' SEBASTIAO FRANCISCO DE QUEIROZ',
 'DFA-',
 'DFA-\n12',
 ' Assessor',
 'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil',
 'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades',
 'Casa Civil, da Governadoria do Distrito Federal',
 'Governadoria do Distrito Federal',
 'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do S

In [122]:
anotators = df.anotador_ent.to_list()
anotators

['pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'pedro_henrique',
 'matheus_stauffer',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'DODFMiner',
 'DODFMiner',
 'matheus_stauffer',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'pedro_henrique',
 'DODFMiner',
 'pedro_henrique',
 'pedro_henrique',


In [123]:
off_sem_duplicata = np.unique(df.offset.to_numpy())

In [124]:
off_sem_duplicata

array([   304,    305,    312, ..., 644348, 644353, 644380])

In [125]:
map_duplicata = {}
for i in off_sem_duplicata:
    map_duplicata[i] = []
for idx, offset in enumerate(offsets):
    map_duplicata[offset].append(idx)

`map_duplicata` é um dicionário cujo objetivo é dispor em uma estrutura de dados todas as ocorrências de um mesmo offset. Para tanto, mapeamos os offsets e linhas do dataframe original. As chaves são todos os offsets únicos, e os valores são listas contendo as linhas onde aquele offset ocorre. Com esse mapa, poderemos iterar entre os valores e adotar as diferentes métricas que quisermos.

In [126]:
map_duplicata

{304: [15160],
 305: [1494],
 312: [9320, 9443, 9566, 9677],
 314: [15161],
 315: [1500],
 323: [15162],
 338: [9321, 9444, 9567, 9678],
 356: [1495],
 359: [12575],
 363: [9322, 9445, 9568, 9679, 15163],
 364: [9323, 9446, 9569, 9680],
 369: [12576],
 378: [12577],
 407: [9324, 9447, 9570, 9681],
 414: [15164],
 424: [15165],
 430: [1496],
 435: [9325, 9448, 9571, 9682],
 439: [12578],
 440: [1497],
 443: [15166],
 445: [9326, 9449, 9572, 9683],
 446: [9327, 9450, 9573, 9684],
 449: [12579],
 451: [1498],
 465: [12580],
 466: [1499],
 468: [9328, 9451, 9574, 9685],
 502: [1446],
 508: [1447],
 522: [9329, 9452, 9575, 9686],
 556: [9389, 9512, 9746, 9776],
 566: [1448],
 577: [9390, 9513, 9747, 9777, 15167],
 588: [9391, 9514, 9748, 9778],
 594: [1449],
 612: [15152],
 622: [15153],
 632: [9392, 9515, 9749, 9779],
 641: [9393, 9516, 9750, 9780],
 663: [15154],
 664: [15155],
 671: [9394, 9517, 9751, 9781],
 681: [1450],
 692: [9395, 9518, 9752, 9782],
 694: [12581],
 703: [15156],
 715

In [132]:
len(map_duplicata)

15566

In [127]:
map_duplicata[312]

[9320, 9443, 9566, 9677]

Para facilitar a consulta aos dados, vamos criar uma lista `data` contendo tuplas da forma `(length, anotador_ent, texto)`. 

In [128]:
data = []
for idx, _ in enumerate(offsets):
    data.append((lengths[idx], anotators[idx], texts[idx]))

Acessamos os elementos via os índices mapeados por `map_duplicata`.

In [131]:
data[9566]

(242,
 'pedro_henrique',
 'EXONERAR, por estar sendo nomeada para outro cargo, ANDRESSA DA COSTA LAN-\nZELLOTTI do Cargo de Natureza Especial, Simbolo CNE-05, de Assessor Especial, da Assessoria \nEspecial, da Assessoria Internacional, da Governadoria do Distrito Federal')

Vamos pegar um exemplo onde as anotações no mesmo offset são distintas:

In [133]:
map_duplicata[205078]

[2, 3, 11, 12, 20, 21, 37, 38]

In [134]:
data[2]

(4, 'DODFMiner', 'DFA-')

In [135]:
data[3]

(7, 'pedro_henrique', 'DFA-\n12')

In [140]:
maior_tam = 0
for i in map_duplicata[205078]:
    # quero identificar o maior length dentre as duplicatas
    if data[i][0] > maior_tam:
        maior_tam = data[i][0]
maior_tam

7

Uma vez que temos as anotações e os tamanhos, entra a métrica em si.

<img src="jaccard.png" alt="Drawing" style="width: 600px;"/>

Fonte da imagem: https://medium.com/data-science-bootcamp/understand-jaccard-index-jaccard-similarity-in-minutes-25a703fbf9d7

Vamos comparar duas (ou mais #TODO: p/+ de 2 duas, possivelmente fazer uma média entre os valores de agreement/dif) strings. Nessa primeira iteração, vamos adotar uma abordagem caractere a caractere. Pro nosso domínio de dados, acredito que faça mais sentido pensar na união como a string com maior tamanho (#TODO: ver outras possibilidades). Contaríamos o número de caracteres que diferem entre as strings e o valor `tam_maior_str - diferença` seria a interseção da fórmula.

In [162]:
str1, str2 = data[2][2], data[3][2]

In [163]:
str1, str2

('DFA-', 'DFA-\n12')

In [166]:
diferenca_char = 0
for idx, _ in enumerate(str2):
    try:
        # TODO: pensar em strings com shift de 1 char. mesmo conteúdo, mas
        # entraria na condição
        if str2[idx] != str1[idx]:
            diferenca_char += 1
    except:
        print(f'idx: {idx}, len: {len(str2)}')
        aaaa = len(str2) - idx
        print(f'entrei aqui: {aaaa}')
        diferenca_char += aaaa
        break
diferenca_char

idx: 4, len: 7
entrei aqui: 3


3

In [170]:
agreement = abs(len(str2) - diferenca_char)/len(str2)
agreement

0.5714285714285714

Ou, pensando em diferença:

In [171]:
dif = 1 - agreement
dif

0.4285714285714286